In [ ]:
import vk_api
import requests
import pandas as pd
import time
import re
import json
from io import StringIO

In [ ]:
CLIENT_ID = '7474646' #ID приложения
link = 'https://oauth.vk.com/authorize?client_id=' + CLIENT_ID + '&scope=friends&response_type=token&v=5.103'
link

In [ ]:
base_url = "https://api.vk.com/method/%s"
access_token = '4d59b687a2df8f008782581d7c549b655433eb99187372455c3fabe9567a4a3b74091fd30a2615b96750f' 

def call(url, params):
    return requests.get(url, params=params)

In [ ]:
schools_df = pd.read_csv('schools1.csv')
schools_df.head()

In [ ]:
schools_dict = {}
#left = len(schools_df)

for school in schools_df['Образовательное учреждение']:
    school_is_found = True
    params = {
        'q': school,
        'city_id': 1,
        'count': 1,
        'v': '5.103'
    }
            
    params = dict(params, **{
            "access_token": access_token
        })
    try:
        vk_school = call(base_url % "database.getSchools", params).json()
        check = vk_school['response']
    except:
        time.sleep(3) 
        vk_school = call(base_url % "database.getSchools", params).json()
    #print(vk_school)
    if vk_school['response']['count'] == 0:
        #print(school)
        number = re.findall('№[0-9]*', school, flags=0)
        if len(number) == 0:
            school_is_found = False
            #print('School has no number')
        else:
            params['q'] = number[0]
            try:
                vk_school = call(base_url % "database.getSchools", params).json()
                check = vk_school['response']
            except:
                time.sleep(3) 
                vk_school = call(base_url % "database.getSchools", params).json()
            #print(vk_school)
            if vk_school['response']['count'] == 0:
                school_is_found = False
    
    if school_is_found:
        schools_dict[school] = vk_school['response']['items'][0]['id']
    
    time.sleep(0.5) 